This Notebook contains an example of using the given DVIP implementation over the considered UCI datasets.

In [1]:
# Notebook reload options
%load_ext autoreload
%autoreload 2

In [2]:

# Global Imports
import torch
from torch.utils.data import DataLoader
import sys

# Local Imports
sys.path.append("..")
from src.dvip import DVIP_Base
from src.likelihood import Gaussian
from src.generative_functions import SimplerBayesLinear
from src.layers_init import init_layers
from utils.dataset import Boston_Dataset
from utils.metrics import MetricsRegression
from utils.pytorch_learning import fit, score


Theese are the parameters that determine the experiments. Most of these values are set to their default value. Please refer to `utils/process_flass.py` to know more about each parameter. Remarkably:

- `vip_layers`: must be specified in a vector. An unique integer corresponds to the given number of layers all of them with the data dimension as width. In order to specify the width of each layer, the vector is used. I.e, [30, 10, 1] means the first layer contains 30 units, the second 10 and the last 1 (must match the target dimensionality).
- `bnn_structure`: Specifies the width of the inner layers of the prior BNN.

In [3]:
params = {"vip_layers": [1],
        "genf": "BNN",
        "regression_coeffs": 20,
        "bnn_structure": [10, 10],
        "bnn_inner_dim": 100,
        "bnn_layer": SimplerBayesLinear,
        "activation": torch.tanh,
        "device": "cpu",
        "dtype": torch.float64,
        "seed": 2147483647,
        "fix_prior_noise": True,
        "genf_full_output": False,
        "final_layer_mu": 0,
        "inner_layers_mu": 0,
        "final_layer_sqrt": 1,
        "inner_layers_sqrt": 1e-5,
        "inner_layers_noise": -5,
        "final_layer_noise": None,
        "dropout": 0.0,
        "prior_kl": False,
        "zero_mean_prior": False,
        "input_prop": True,
        "bb_alpha": 0.5
}

Set seed for reproductibility.

In [4]:
torch.manual_seed(params["seed"]);

Load Dataset and desired split.

In [5]:
dataset = Boston_Dataset()
split = 0
train_dataset, train_test_dataset, test_dataset = dataset.get_split(test_size = 0.1, 
                                                                    seed = params["seed"] + split)

Number of samples:  455
Input dimension:  13
Label dimension:  1
Labels mean value:  [[22.51604396]]
Labels standard deviation:  [[9.23007422]]


Create Data loaders for training and test partitions.

In [6]:
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
train_test_loader = DataLoader(train_test_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [7]:
layers = init_layers(train_dataset.inputs, dataset.output_dim, **params)

Layer 0: 13->1 MF: None


Define a likelihood.

In [8]:
likelihood = Gaussian(dtype = params["dtype"], device = params["device"])

Define DVIP model

In [9]:
dvip = DVIP_Base(
    likelihood,
    layers,
    len(train_dataset),
    bb_alpha=params["bb_alpha"],
    num_samples=1,
    y_mean=train_dataset.targets_mean,
    y_std=train_dataset.targets_std,
    dtype=params["dtype"],
    device=params["device"]
)

Start training.

In [10]:
# Define optimizer and compile model
opt = torch.optim.Adam(dvip.parameters(), lr=0.001)


fit(
    dvip,
    train_loader,
    opt,
    use_tqdm=True,
    return_loss=False,
    iterations=150_000,
    device=params["device"],
)


Training : 100%|██████████| 150000/150000 [13:57<00:00, 179.08 iteration/s]


[]

Compute test metrics.

In [11]:
metrics = MetricsRegression

# Set the number of test samples to generate
dvip.num_samples = 100

# Test the model
train_metrics = score(dvip, train_test_loader, MetricsRegression, use_tqdm = True, device=params["device"])
test_metrics = score(dvip, test_loader, MetricsRegression, use_tqdm=True, device=params["device"])

print("TEST RESULTS: ")
for k, v in test_metrics.items():
    print("\t - {}: {}".format(k, v))

Evaluating : 100%|██████████| 1/1 [00:00<00:00,  7.99iteration/s]

TEST RESULTS: 
	 - LOSS: 86.90116882324219
	 - RMSE: 5.8104681968688965
	 - NLL: 2.8939199447631836
	 - CRPS: 2.52249813079834
